TARGET LABEL: Helpful Votes
SCALER: STANDARD SCALER
ANN'S : 
    Combination 1: 7-100-100-1
    
    Combination 2:  7-4-4-1

    Combination 3:  7-50-100-200-100-50-20-1

    Combination 4: 7-50-50-11

    Combination 5: 7-20-20-11
    
Standardizing the target variable to test the difference in MSE

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv(r"/Users/t_velpac/mission/WorkingCopy/final_csv's/Features_Sentiments.csv")
dataset = dataset.drop(columns=['Date', 'Z_Score_HelpfulVotes', 'Z_Score_Words', 'Z_Score_Paragraphs'])
dataset = dataset[['Stars','Words', 'Paragraphs','Sentiment_Polarity', 'No.break tags','Percentage_Upper_Case','Percentage_Lower_Case','Avg_len_paragraph_per_review','Helpful_Votes']]
dataset.head()

,Stars,Words,Paragraphs,Sentiment_Polarity,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Helpful_Votes
0,4,268,8,0.132353,14,3,90,166.125000,146
1,1,130,2,0.110238,2,2,94,352.500000,80
2,1,110,1,0.077232,0,3,90,556.000000,80
3,5,110,3,0.125938,4,3,92,188.666667,1563
4,1,39,1,-0.180556,0,3,96,216.000000,53


In [3]:
"""Scaling the data using StandardScaler from sklearn package"""
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
dataset = sc.fit_transform(dataset)

""" Splitting the independent variables from the dependent variable """
X = dataset[:,0:-1]
y = dataset[:,-1]

/anaconda3/envs/WorkingCopy/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/WorkingCopy/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [4]:
"""Splitting the data into training data and testing data"""
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=0)

In [5]:
""" Building a 3 layer ANN with high number of nodes """

regressor = Sequential()
regressor.add(Dense(100, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor.add(Dense(100, kernel_initializer = 'normal', activation = 'relu'))
regressor.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
156392/156392 [==============================] - 26s 165us/step - loss: 0.9499
Epoch 2/100
156392/156392 [==============================] - 27s 173us/step - loss: 0.9494
Epoch 3/100
156392/156392 [==============================] - 26s 164us/step - loss: 0.9494
Epoch 4/100
156392/156392 [==============================] - 27s 173us/step - loss: 0.9488
Epoch 5/100
156392/156392 [==============================] - 28s 182us/step - loss: 0.9494
Epoch 6/100
156392/156392 [==============================] - 28s 178us/step - loss: 0.9492
Epoch 7/100
156392/156392 [==============================] - 26s 164us/step - loss: 0.9491
Epoch 8/100
156392/156392 [==============================] - 26s 169us/step - loss: 0.9492
Epoch 9/100
156392/156392 [==============================] - 29s 186us/step - loss: 0.9489
Epoch 10/100
156392/156392 [==============================] -

156392/156392 [==============================] - 25s 159us/step - loss: 0.9488
Epoch 84/100
156392/156392 [==============================] - 25s 158us/step - loss: 0.9490
Epoch 85/100
156392/156392 [==============================] - 25s 158us/step - loss: 0.9488
Epoch 86/100
156392/156392 [==============================] - 25s 158us/step - loss: 0.9488
Epoch 87/100
156392/156392 [==============================] - 26s 165us/step - loss: 0.9490
Epoch 88/100
156392/156392 [==============================] - 27s 169us/step - loss: 0.9483
Epoch 89/100
156392/156392 [==============================] - 25s 160us/step - loss: 0.9489
Epoch 90/100
156392/156392 [==============================] - 26s 164us/step - loss: 0.9488
Epoch 91/100
156392/156392 [==============================] - 26s 164us/step - loss: 0.9480
Epoch 92/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9488
Epoch 93/100
156392/156392 [==============================] - 25s 159us/step - loss: 0.9491
E

In [6]:
""" Predicting the values for Helpful Votes on the test data """
y_pred = regressor.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred = y_pred.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred}
y_compare = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare['Mean Squared Error'] = (np.diff(y_compare.values) ** 2)
y_compare['Mean Squared Error'] = np.mean(y_compare['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare['Root Mean Squared Error'] = y_compare['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare['Mean Abs. Error'] = np.mean(abs(y_compare['Actual Values'] - y_compare['Predicted Values']))

y_compare

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,0.154273,1.194937,1.093132,0.065242
1,-0.033284,-0.014112,1.194937,1.093132,0.065242
2,-0.033284,-0.022801,1.194937,1.093132,0.065242
3,-0.033284,0.028949,1.194937,1.093132,0.065242
4,-0.033284,0.023539,1.194937,1.093132,0.065242
5,-0.033284,-0.027509,1.194937,1.093132,0.065242
6,-0.009864,0.056030,1.194937,1.093132,0.065242
7,0.044782,0.010480,1.194937,1.093132,0.065242
8,-0.033284,-0.033540,1.194937,1.093132,0.065242
9,-0.033284,0.001738,1.194937,1.093132,0.065242


In [7]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train = regressor.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train = y_pred_train.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train }
y_compare_train = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train['Mean Squared Error'] = (np.diff(y_compare_train.values) ** 2)
y_compare_train['Mean Squared Error'] = np.mean(y_compare_train['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train['Root Mean Squared Error'] = y_compare_train['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train['Mean Abs. Error'] = np.mean(abs(y_compare_train['Actual Values(Training)'] - y_compare_train['Predicted Values(Training)']))

y_compare_train

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,-0.003838,0.947945,0.973624,0.065571
1,-0.033284,0.155105,0.947945,0.973624,0.065571
2,-0.033284,-0.048801,0.947945,0.973624,0.065571
3,-0.033284,-0.003673,0.947945,0.973624,0.065571
4,-0.033284,-0.007282,0.947945,0.973624,0.065571
5,0.138462,-0.003391,0.947945,0.973624,0.065571
6,-0.025478,0.006524,0.947945,0.973624,0.065571
7,-0.033284,0.001201,0.947945,0.973624,0.065571
8,-0.033284,-0.043359,0.947945,0.973624,0.065571
9,-0.033284,-0.010723,0.947945,0.973624,0.065571


In [8]:
""" Building a 3 layer ANN with 2 layers of 4 nodes """

regressor_1 = Sequential()
regressor_1.add(Dense(4, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_1.add(Dense(4, kernel_initializer = 'normal', activation = 'relu'))
regressor_1.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_1.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_1.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 25s 162us/step - loss: 0.9481
Epoch 2/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9481
Epoch 3/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9479
Epoch 4/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9479
Epoch 5/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9484
Epoch 6/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9481
Epoch 7/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9490
Epoch 8/100
156392/156392 [==============================] - 25s 160us/step - loss: 0.9482
Epoch 9/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9485
Epoch 10/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9485
Epoch 11/100
156392/156392 [==============================] - 26s 164us/step - loss: 0.94

156392/156392 [==============================] - 25s 161us/step - loss: 0.9484
Epoch 91/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9483
Epoch 92/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9483
Epoch 93/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9481
Epoch 94/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9480
Epoch 95/100
156392/156392 [==============================] - 25s 162us/step - loss: 0.9482
Epoch 96/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9482
Epoch 97/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9483
Epoch 98/100
156392/156392 [==============================] - 25s 162us/step - loss: 0.9481
Epoch 99/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9482
Epoch 100/100
156392/156392 [==============================] - 25s 161us/step - loss: 0.9483


In [9]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_1 = regressor_1.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_1 = y_pred_1.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_1}
y_compare_1 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_1['Mean Squared Error'] = (np.diff(y_compare_1.values) ** 2)
y_compare_1['Mean Squared Error'] = np.mean(y_compare_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_1['Root Mean Squared Error'] = y_compare_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_1['Mean Abs. Error'] = np.mean(abs(y_compare_1['Actual Values'] - y_compare_1['Predicted Values']))

y_compare_1

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,0.102819,1.194787,1.093063,0.059321
1,-0.033284,-0.031308,1.194787,1.093063,0.059321
2,-0.033284,-0.031713,1.194787,1.093063,0.059321
3,-0.033284,0.039178,1.194787,1.093063,0.059321
4,-0.033284,-0.008827,1.194787,1.093063,0.059321
5,-0.033284,-0.032718,1.194787,1.093063,0.059321
6,-0.009864,0.057289,1.194787,1.093063,0.059321
7,0.044782,0.000485,1.194787,1.093063,0.059321
8,-0.033284,-0.039678,1.194787,1.093063,0.059321
9,-0.033284,-0.007575,1.194787,1.093063,0.059321


In [10]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_1 = regressor_1.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_1 = y_pred_train_1.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_1 }
y_compare_train_1 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_1['Mean Squared Error'] = (np.diff(y_compare_train_1.values) ** 2)
y_compare_train_1['Mean Squared Error'] = np.mean(y_compare_train_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_1['Root Mean Squared Error'] = y_compare_train_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_1['Mean Abs. Error'] = np.mean(abs(y_compare_train_1['Actual Values(Training)'] - y_compare_train_1['Predicted Values(Training)']))

y_compare_train_1

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,-0.022932,0.94733,0.973309,0.05954
1,-0.033284,0.219583,0.94733,0.973309,0.05954
2,-0.033284,-0.023261,0.94733,0.973309,0.05954
3,-0.033284,-0.020983,0.94733,0.973309,0.05954
4,-0.033284,-0.025087,0.94733,0.973309,0.05954
5,0.138462,-0.013523,0.94733,0.973309,0.05954
6,-0.025478,0.010684,0.94733,0.973309,0.05954
7,-0.033284,-0.008159,0.94733,0.973309,0.05954
8,-0.033284,-0.040391,0.94733,0.973309,0.05954
9,-0.033284,-0.020028,0.94733,0.973309,0.05954


In [11]:
""" Building a deep and wide ANN with high number of nodes """

from keras.layers.advanced_activations import LeakyReLU
regressor_2 = Sequential()
regressor_2.add(Dense(50, kernel_initializer = 'normal',input_dim = 8))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(200, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(50, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(20, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_2.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_2.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 45s 288us/step - loss: 0.9518
Epoch 2/100
156392/156392 [==============================] - 43s 272us/step - loss: 0.9504
Epoch 3/100
156392/156392 [==============================] - 40s 255us/step - loss: 0.9508
Epoch 4/100
156392/156392 [==============================] - 41s 261us/step - loss: 0.9502
Epoch 5/100
156392/156392 [==============================] - 40s 256us/step - loss: 0.9508
Epoch 6/100
156392/156392 [==============================] - 42s 267us/step - loss: 0.9513
Epoch 7/100
156392/156392 [==============================] - 40s 256us/step - loss: 0.9507
Epoch 8/100
156392/156392 [==============================] - 40s 258us/step - loss: 1.0432
Epoch 9/100
156392/156392 [==============================] - 40s 253us/step - loss: 5.8128
Epoch 10/100
156392/156392 [==============================] - 40s 255us/step - loss: 0.9521
Epoch 11/100
156392/156392 [==============================] - 41s 263us/step - loss: 4.09

156392/156392 [==============================] - 42s 271us/step - loss: 12.4705
Epoch 89/100
156392/156392 [==============================] - 46s 291us/step - loss: 191.0304
Epoch 90/100
156392/156392 [==============================] - 45s 291us/step - loss: 0.9876
Epoch 91/100
156392/156392 [==============================] - 43s 275us/step - loss: 381.6838
Epoch 92/100
156392/156392 [==============================] - 41s 265us/step - loss: 39708.7179
Epoch 93/100
156392/156392 [==============================] - 43s 277us/step - loss: 27741.4051
Epoch 94/100
156392/156392 [==============================] - 41s 259us/step - loss: 115.0356
Epoch 95/100
156392/156392 [==============================] - 40s 258us/step - loss: 0.9624
Epoch 96/100
156392/156392 [==============================] - 50s 322us/step - loss: 31.9698
Epoch 97/100
156392/156392 [==============================] - 42s 266us/step - loss: 0.9867
Epoch 98/100
156392/156392 [==============================] - 48s 310us/step 

In [12]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_2 = regressor_2.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_2 = y_pred_2.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_2}
y_compare_2 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_2['Mean Squared Error'] = (np.diff(y_compare_2.values) ** 2)
y_compare_2['Mean Squared Error'] = np.mean(y_compare_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_2['Root Mean Squared Error'] = y_compare_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_2['Mean Abs. Error'] = np.mean(abs(y_compare_2['Actual Values'] - y_compare_2['Predicted Values']))

y_compare_2

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,0.425378,1.202088,1.096398,0.088634
1,-0.033284,-0.029921,1.202088,1.096398,0.088634
2,-0.033284,0.014156,1.202088,1.096398,0.088634
3,-0.033284,0.066655,1.202088,1.096398,0.088634
4,-0.033284,0.027055,1.202088,1.096398,0.088634
5,-0.033284,-0.034553,1.202088,1.096398,0.088634
6,-0.009864,0.066909,1.202088,1.096398,0.088634
7,0.044782,-0.067865,1.202088,1.096398,0.088634
8,-0.033284,-0.011802,1.202088,1.096398,0.088634
9,-0.033284,0.024127,1.202088,1.096398,0.088634


In [13]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_2 = regressor_2.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_2 = y_pred_train_2.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_2 }
y_compare_train_2 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_2['Mean Squared Error'] = (np.diff(y_compare_train_2.values) ** 2)
y_compare_train_2['Mean Squared Error'] = np.mean(y_compare_train_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_2['Root Mean Squared Error'] = y_compare_train_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_2['Mean Abs. Error'] = np.mean(abs(y_compare_train_2['Actual Values(Training)'] - y_compare_train_2['Predicted Values(Training)']))

y_compare_train_2

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,-0.036211,1.48197,1.217362,0.091297
1,-0.033284,0.369952,1.48197,1.217362,0.091297
2,-0.033284,0.048567,1.48197,1.217362,0.091297
3,-0.033284,-0.040913,1.48197,1.217362,0.091297
4,-0.033284,-0.030753,1.48197,1.217362,0.091297
5,0.138462,-0.058902,1.48197,1.217362,0.091297
6,-0.025478,-0.035713,1.48197,1.217362,0.091297
7,-0.033284,0.020716,1.48197,1.217362,0.091297
8,-0.033284,-0.012636,1.48197,1.217362,0.091297
9,-0.033284,-0.048053,1.48197,1.217362,0.091297


In [14]:
""" Building a 3 layer ANN with 2 layers of 50 nodes """

regressor_3 = Sequential()
regressor_3.add(Dense(50, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_3.add(Dense(50, kernel_initializer = 'normal', activation = 'relu'))
regressor_3.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_3.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_3.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 32s 203us/step - loss: 0.9504
Epoch 2/100
156392/156392 [==============================] - 27s 175us/step - loss: 0.9494
Epoch 3/100
156392/156392 [==============================] - 26s 169us/step - loss: 0.9495
Epoch 4/100
156392/156392 [==============================] - 26s 168us/step - loss: 0.9492
Epoch 5/100
156392/156392 [==============================] - 31s 196us/step - loss: 0.9502
Epoch 6/100
156392/156392 [==============================] - 30s 193us/step - loss: 0.9492
Epoch 7/100
156392/156392 [==============================] - 31s 197us/step - loss: 0.9502
Epoch 8/100
156392/156392 [==============================] - 28s 177us/step - loss: 0.9500
Epoch 9/100
156392/156392 [==============================] - 27s 175us/step - loss: 0.9509
Epoch 10/100
156392/156392 [==============================] - 28s 182us/step - loss: 0.9497
Epoch 11/100
156392/156392 [==============================] - 28s 180us/step - loss: 0.94

156392/156392 [==============================] - 26s 167us/step - loss: 0.9499
Epoch 90/100
156392/156392 [==============================] - 26s 169us/step - loss: 0.9496
Epoch 91/100
156392/156392 [==============================] - 26s 167us/step - loss: 0.9501
Epoch 92/100
156392/156392 [==============================] - 26s 168us/step - loss: 0.9493
Epoch 93/100
156392/156392 [==============================] - 26s 168us/step - loss: 0.9500
Epoch 94/100
156392/156392 [==============================] - 27s 173us/step - loss: 0.9494
Epoch 95/100
156392/156392 [==============================] - 27s 173us/step - loss: 0.9492
Epoch 96/100
156392/156392 [==============================] - 27s 171us/step - loss: 0.9503
Epoch 97/100
156392/156392 [==============================] - 26s 168us/step - loss: 0.9495
Epoch 98/100
156392/156392 [==============================] - 26s 168us/step - loss: 0.9486
Epoch 99/100
156392/156392 [==============================] - 27s 172us/step - loss: 0.9493
E

In [15]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_3 = regressor_3.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_3 = y_pred_3.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_3}
y_compare_3 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_3['Mean Squared Error'] = (np.diff(y_compare_3.values) ** 2)
y_compare_3['Mean Squared Error'] = np.mean(y_compare_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_3['Root Mean Squared Error'] = y_compare_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_3['Mean Abs. Error'] = np.mean(abs(y_compare_3['Actual Values'] - y_compare_3['Predicted Values']))

y_compare_3

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,0.024017,1.195705,1.093483,0.043506
1,-0.033284,-0.020654,1.195705,1.093483,0.043506
2,-0.033284,-0.020851,1.195705,1.093483,0.043506
3,-0.033284,-0.015164,1.195705,1.093483,0.043506
4,-0.033284,-0.023651,1.195705,1.093483,0.043506
5,-0.033284,-0.036670,1.195705,1.093483,0.043506
6,-0.009864,-0.005871,1.195705,1.093483,0.043506
7,0.044782,-0.008417,1.195705,1.093483,0.043506
8,-0.033284,-0.031345,1.195705,1.093483,0.043506
9,-0.033284,-0.014119,1.195705,1.093483,0.043506


In [16]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_3 = regressor_3.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_3 = y_pred_train_3.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_3 }
y_compare_train_3 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_3['Mean Squared Error'] = (np.diff(y_compare_train_3.values) ** 2)
y_compare_train_3['Mean Squared Error'] = np.mean(y_compare_train_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_3['Root Mean Squared Error'] = y_compare_train_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_3['Mean Abs. Error'] = np.mean(abs(y_compare_train_3['Actual Values(Training)'] - y_compare_train_3['Predicted Values(Training)']))

y_compare_train_3

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,-0.018735,0.948343,0.973829,0.04357
1,-0.033284,0.136987,0.948343,0.973829,0.04357
2,-0.033284,-0.030241,0.948343,0.973829,0.04357
3,-0.033284,-0.020370,0.948343,0.973829,0.04357
4,-0.033284,-0.023476,0.948343,0.973829,0.04357
5,0.138462,-0.019862,0.948343,0.973829,0.04357
6,-0.025478,-0.028275,0.948343,0.973829,0.04357
7,-0.033284,-0.029719,0.948343,0.973829,0.04357
8,-0.033284,-0.044308,0.948343,0.973829,0.04357
9,-0.033284,-0.020385,0.948343,0.973829,0.04357


In [17]:
""" Building a 3 layer ANN with 2 layers of 20 nodes """

regressor_4 = Sequential()
regressor_4.add(Dense(20, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_4.add(Dense(20, kernel_initializer = 'normal', activation = 'relu'))
regressor_4.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_4.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_4.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 28s 176us/step - loss: 0.9490
Epoch 2/100
156392/156392 [==============================] - 27s 172us/step - loss: 0.9496
Epoch 3/100
156392/156392 [==============================] - 27s 172us/step - loss: 0.9493
Epoch 4/100
156392/156392 [==============================] - 27s 173us/step - loss: 0.9492
Epoch 5/100
156392/156392 [==============================] - 27s 174us/step - loss: 0.9489
Epoch 6/100
156392/156392 [==============================] - 27s 170us/step - loss: 0.9487
Epoch 7/100
156392/156392 [==============================] - 27s 170us/step - loss: 0.9487
Epoch 8/100
156392/156392 [==============================] - 27s 170us/step - loss: 0.9490
Epoch 9/100
156392/156392 [==============================] - 27s 170us/step - loss: 0.9488
Epoch 10/100
156392/156392 [==============================] - 27s 171us/step - loss: 0.9492
Epoch 11/100
156392/156392 [==============================] - 27s 171us/step - loss: 0.94

156392/156392 [==============================] - 28s 180us/step - loss: 0.9488
Epoch 90/100
156392/156392 [==============================] - 28s 180us/step - loss: 0.9488
Epoch 91/100
156392/156392 [==============================] - 29s 186us/step - loss: 0.9487
Epoch 92/100
156392/156392 [==============================] - 28s 181us/step - loss: 0.9490
Epoch 93/100
156392/156392 [==============================] - 29s 187us/step - loss: 0.9489
Epoch 94/100
156392/156392 [==============================] - 29s 188us/step - loss: 0.9488
Epoch 95/100
156392/156392 [==============================] - 28s 180us/step - loss: 0.9490
Epoch 96/100
156392/156392 [==============================] - 30s 189us/step - loss: 0.9485
Epoch 97/100
156392/156392 [==============================] - 29s 183us/step - loss: 0.9491
Epoch 98/100
156392/156392 [==============================] - 29s 185us/step - loss: 0.9491
Epoch 99/100
156392/156392 [==============================] - 29s 185us/step - loss: 0.9489
E

In [18]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_4 = regressor_4.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_4 = y_pred_4.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_4}
y_compare_4 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_4['Mean Squared Error'] = (np.diff(y_compare_4.values) ** 2)
y_compare_4['Mean Squared Error'] = np.mean(y_compare_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_4['Root Mean Squared Error'] = y_compare_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_4['Mean Abs. Error'] = np.mean(abs(y_compare_4['Actual Values'] - y_compare_4['Predicted Values']))

y_compare_4

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,0.089450,1.195104,1.093208,0.053492
1,-0.033284,-0.022860,1.195104,1.093208,0.053492
2,-0.033284,-0.022507,1.195104,1.093208,0.053492
3,-0.033284,0.029722,1.195104,1.093208,0.053492
4,-0.033284,-0.011036,1.195104,1.093208,0.053492
5,-0.033284,-0.035343,1.195104,1.093208,0.053492
6,-0.009864,0.027781,1.195104,1.093208,0.053492
7,0.044782,-0.019302,1.195104,1.093208,0.053492
8,-0.033284,-0.032233,1.195104,1.093208,0.053492
9,-0.033284,-0.015001,1.195104,1.093208,0.053492


In [19]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_4 = regressor_4.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_4 = y_pred_train_4.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_4 }
y_compare_train_4 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_4['Mean Squared Error'] = (np.diff(y_compare_train_4.values) ** 2)
y_compare_train_4['Mean Squared Error'] = np.mean(y_compare_train_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_4['Root Mean Squared Error'] = y_compare_train_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_4['Mean Abs. Error'] = np.mean(abs(y_compare_train_4['Actual Values(Training)'] - y_compare_train_4['Predicted Values(Training)']))

y_compare_train_4

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,-0.021811,0.947818,0.973559,0.053749
1,-0.033284,0.135377,0.947818,0.973559,0.053749
2,-0.033284,-0.012768,0.947818,0.973559,0.053749
3,-0.033284,-0.023410,0.947818,0.973559,0.053749
4,-0.033284,-0.024316,0.947818,0.973559,0.053749
5,0.138462,-0.024859,0.947818,0.973559,0.053749
6,-0.025478,0.005345,0.947818,0.973559,0.053749
7,-0.033284,0.000192,0.947818,0.973559,0.053749
8,-0.033284,-0.041372,0.947818,0.973559,0.053749
9,-0.033284,-0.027711,0.947818,0.973559,0.053749


In [21]:
""" Building an ANN with 8 layers of 200 nodes """

regressor_5 = Sequential()
regressor_5.add(Dense(200, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_5.add(Dense(200, kernel_initializer = 'normal', activation = 'relu'))
regressor_5.add(Dense(200, kernel_initializer = 'normal', activation = 'relu'))
regressor_5.add(Dense(200, kernel_initializer = 'normal', activation = 'relu'))
regressor_5.add(Dense(200, kernel_initializer = 'normal', activation = 'relu'))
regressor_5.add(Dense(200, kernel_initializer = 'normal', activation = 'relu'))
regressor_5.add(Dense(200, kernel_initializer = 'normal', activation = 'relu'))
regressor_5.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_5.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_5.fit(X_train, y_train, batch_size=10, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 34s 216us/step - loss: 0.9512
Epoch 2/100
156392/156392 [==============================] - 33s 209us/step - loss: 0.9515
Epoch 3/100
156392/156392 [==============================] - 32s 205us/step - loss: 0.9511
Epoch 4/100
156392/156392 [==============================] - 32s 203us/step - loss: 0.9505
Epoch 5/100
156392/156392 [==============================] - 31s 200us/step - loss: 0.9505
Epoch 6/100
156392/156392 [==============================] - 31s 199us/step - loss: 0.9516
Epoch 7/100
156392/156392 [==============================] - 31s 200us/step - loss: 0.9513
Epoch 8/100
156392/156392 [==============================] - 35s 226us/step - loss: 0.9511
Epoch 9/100
156392/156392 [==============================] - 33s 213us/step - loss: 0.9511
Epoch 10/100
156392/156392 [==============================] - 33s 214us/step - loss: 0.9510
Epoch 11/100
156392/156392 [==============================] - 33s 212us/step - loss: 0.95

156392/156392 [==============================] - 34s 215us/step - loss: 0.9509
Epoch 90/100
156392/156392 [==============================] - 34s 216us/step - loss: 0.9509
Epoch 91/100
156392/156392 [==============================] - 34s 214us/step - loss: 0.9509
Epoch 92/100
156392/156392 [==============================] - 33s 214us/step - loss: 0.9509
Epoch 93/100
156392/156392 [==============================] - 34s 215us/step - loss: 0.9509
Epoch 94/100
156392/156392 [==============================] - 34s 220us/step - loss: 0.9509
Epoch 95/100
156392/156392 [==============================] - 35s 226us/step - loss: 0.9509
Epoch 96/100
156392/156392 [==============================] - 34s 218us/step - loss: 0.9509
Epoch 97/100
156392/156392 [==============================] - 36s 231us/step - loss: 0.9509
Epoch 98/100
156392/156392 [==============================] - 34s 220us/step - loss: 0.9509
Epoch 99/100
156392/156392 [==============================] - 35s 222us/step - loss: 0.9509
E

In [22]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_5 = regressor_5.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_5 = y_pred_5.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_5}
y_compare_5 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_5['Mean Squared Error'] = (np.diff(y_compare_5.values) ** 2)
y_compare_5['Mean Squared Error'] = np.mean(y_compare_5['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_5['Root Mean Squared Error'] = y_compare_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_5['Mean Abs. Error'] = np.mean(abs(y_compare_4['Actual Values'] - y_compare_4['Predicted Values']))

y_compare_5

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,0.005642,1.196917,1.093208,0.053492
1,-0.033284,0.005642,1.196917,1.093208,0.053492
2,-0.033284,0.005642,1.196917,1.093208,0.053492
3,-0.033284,0.005642,1.196917,1.093208,0.053492
4,-0.033284,0.005642,1.196917,1.093208,0.053492
5,-0.033284,0.005642,1.196917,1.093208,0.053492
6,-0.009864,0.005642,1.196917,1.093208,0.053492
7,0.044782,0.005642,1.196917,1.093208,0.053492
8,-0.033284,0.005642,1.196917,1.093208,0.053492
9,-0.033284,0.005642,1.196917,1.093208,0.053492


In [23]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_5 = regressor_5.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_5 = y_pred_train_5.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_5 }
y_compare_train_5 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_5['Mean Squared Error'] = (np.diff(y_compare_train_5.values) ** 2)
y_compare_train_5['Mean Squared Error'] = np.mean(y_compare_train_5['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_5['Root Mean Squared Error'] = y_compare_train_5['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_5['Mean Abs. Error'] = np.mean(abs(y_compare_train_5['Actual Values(Training)'] - y_compare_train_5['Predicted Values(Training)']))

y_compare_train_5

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.033284,0.005642,0.950809,0.975095,0.062093
1,-0.033284,0.005642,0.950809,0.975095,0.062093
2,-0.033284,0.005642,0.950809,0.975095,0.062093
3,-0.033284,0.005642,0.950809,0.975095,0.062093
4,-0.033284,0.005642,0.950809,0.975095,0.062093
5,0.138462,0.005642,0.950809,0.975095,0.062093
6,-0.025478,0.005642,0.950809,0.975095,0.062093
7,-0.033284,0.005642,0.950809,0.975095,0.062093
8,-0.033284,0.005642,0.950809,0.975095,0.062093
9,-0.033284,0.005642,0.950809,0.975095,0.062093
